In [5]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums
from time import sleep
import time
from multiprocessing import Pool

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-01-26 19:21:06.962999


In [6]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
albs = albums(disc)
dutils = discogsUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


# Collections

In [ ]:
%load_ext autoreload
%autoreload
#col = collections(disc)
#col.parseCollections(force=False)
#col.createCollectionDBs()

# Parse Artists (Parallel)

In [7]:
%load_ext autoreload
%autoreload

from random import randint

from artist import artist
from artists import artists
arts = artists(disc)

def parseArtists(modVal, force=False, doExtra=False):
    arts.parseArtistModValFiles(modVal, force=force)
    #sleep(randint(1,10))
    #arts.parseArtistModValExtraFiles(modVal)
    #arts.createArtistModValMetadata(modVal=modVal)
    #arts.createArtistAlbumModValMetadata(modVal=modVal)
    
def parseArtistsParallel(nProcs=3, force=False):
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseArtists, range(100)) 
    #result = pool.map_async(parseArtists, [97, 96, 98, 62])

    while not result.ready():
        if force is True:
            time.sleep(10)
        else:
            time.sleep(1)
    print("")
    return result.get()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
#arts.parseArtistModValFiles(27, force=True)
#arts.parseArtistModValFiles(50, force=True)

In [9]:
start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")
parseArtistsParallel(nProcs=8)
elapsed(start2, cmt2)
elapsed(start, cmt)

Current Time is Sun Jan 26, 2020 19:21:37 for Parsing All Artists
Current Time is Sun Jan 26, 2020 19:21:37 for Parallel Artist Parsing
Parsing Artist Files For ModVal 16
Parsing Artist Files For ModVal 8
Parsing Artist Files For ModVal 12
Parsing Artist Files For ModVal 4
Parsing Artist Files For ModVal 0
Parsing Artist Files For ModVal 24
Parsing Artist Files For ModVal 20
Parsing Artist Files For ModVal 28
  ===> Found 7176 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 5
  ===> Found 7042 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 1
  ===> Found 7067 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 17
  ===> Found 7186 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 13
  ===> Found 7148 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 29
  ===> Found 7157 files, but there are no ne

Parsing Artist Files For ModVal 70
  ===> Found 7064 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 89
  ===> Found 7084 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 78
  ===> Found 7298 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 82
  ===> Found 7081 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 74
  ===> Found 7267 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 93
  ===> Found 7169 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 67
  ===> Found 7038 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 71
  ===> Found 7094 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 86
  ===> Found 7273 files. There are 1 new files to parse.
  ===> Found 7230 files, but there a

# Albums

In [ ]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
from discogsBase import discogs

disc = discogs()
albs = albums(disc)
#albs.parseAlbumModValFiles(modVal=27)

In [ ]:
def parseAlbums(modVal):
    albs.parseAlbumModValFiles(modVal=modVal)
    #albs.createAlbumModValMetadata(modVal)
    #albs.createAlbumModValMetadata(modVal=modVal)

def parseAlbumsParallel(modVals=None):
    if modVals is None:
        modVals = ["NAN"] + list(range(100))
        #modVals = range(100)
    nProcs = 2
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseAlbums, modVals)
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        #print(".",end="")
        time.sleep(10)
    print("")
    
    elapsed(start, cmt)
    return result.get()

In [ ]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
albs = albums(disc)

start,cmt=clock("Parsing All Albums")
start2,cmt2=clock("Parallel Album Parsing")
parseAlbumsParallel()
elapsed(start2, cmt2)
elapsed(start, cmt)